# **Simple Scenario Example - Charleston, USA**

In this notebook we demonstrate the **workflow** on how you can build a simple **FloodAdapt scenario** in Charleston, USA, using the **API**.  

We prepared a database, which you can download from [**here**](link.com). If you worked with the [**Database-Builder notebook**](database_builder.ipynb) beforehand, you can use that database.


In this notebook we will cover the following steps:

1. Create a synthetic **event** 
2. Create a **projection** - Sea level rise (SLR)
3. Create a **measure** and **strategy** - Seawall
4. Create and run a **scenario**
6. Investigate the **output**


## **Step 0**: 📘Import packages

In [ ]:
# Import packages
import contextily as cx
from datetime import datetime
import geopandas as gpd
from pathlib import Path
import rasterio
import rasterio.plot

from flood_adapt.config.sfincs import RiverModel
from flood_adapt.dbs_classes.database import Database
from flood_adapt.objects.events.synthetic import SyntheticEvent 
from flood_adapt.objects.forcing.discharge import DischargeConstant
from flood_adapt.objects.forcing.forcing import ForcingType
from flood_adapt.flood_adapt import FloodAdapt
from flood_adapt.objects.forcing.forcing import (
    ForcingSource,
    ForcingType,)
from flood_adapt.objects.forcing.rainfall import RainfallConstant
from flood_adapt.objects.forcing.time_frame import (
    TimeFrame,
)
from flood_adapt.objects.forcing.timeseries import (
    ShapeType,
    TimeseriesFactory,
)
from flood_adapt.objects.forcing.waterlevels import (
    SurgeModel,
    TideModel,
    WaterlevelSynthetic,
)
from flood_adapt.objects.forcing.wind import WindConstant
from flood_adapt.objects.measures.measures import (
    Elevate,
    FloodWall,
    Measure,
    MeasureType,
    SelectionType,
    HazardMeasure
)
from flood_adapt.objects.projections.projections import Projection, PhysicalProjection, SocioEconomicChange
from flood_adapt.objects.scenarios.scenarios import Scenario
from flood_adapt import Settings
from flood_adapt.objects.strategies.strategies import Strategy
from flood_adapt import unit_system as us

## **Step 1**: 🚀 Getting started - Initiate the database

Let's start with initiating the database and `FloodAdapt` class. 
1. Initiate the database `Settings` class by defining the `database_root` and `database_name`
2. Initiate the `FloodAdapt` class by parsing the `Settings().database_path`

In [ ]:
# Set up the settings for the database
Settings(
    DATABASE_ROOT=Path("../_data/examples").resolve(),
    DATABASE_NAME="charleston_test"
)

# Create the FloodAdapt instance
fa = FloodAdapt(Settings().database_path)

## **Step 2**: 🌊 Events - Create a synthetic Event

Events in FloodAdapt are categorized into different forcings:
1. **Wind**
2. **Rainfall**
3. **Discharge**
4. **Water Level**

If you want to learn more about the individual forcings in FloodAdapt, please go and read the section on [**Events**](https://deltares-research.github.io/FloodAdapt/4_user_guide/events/) in the FloodAdapt documentation.

When creating an event we need to create an `Event` object with the following inputs:  
`name`: The name of the event  
`description` (optional): The description of the event  
`time`: An object that captures the start_time and end_time  
`forcings`: A dictionary the collects the objects for each forcing.  

In this event example we will create an event with the following `forcings`:  
`WindConstant`: Define a value for a constant wind speed (mps) and direction (degrees)  
`RainfallConstant`: Define a value for a constant rainfall (mm/hr)  
`DischargeConstant`: Define the x and y coordinates of the discharge point of the Cooper River and a value for a constant mean discharge (cfs) in the River- and Discharge model (same value)  
`WaterlevelSynthetic SurgeModel`: Define a peak time (h), peak value in (m) and duration (d)  
`WaterlevelSynthetic TideModel`: Define the harmonic amplitude (m), harmonic period (h) and harmonic phase (h)  

For a complete guide on all the possible event inputs and options check out the notebook [**eventblablab**](nolink).

In [ ]:
# Create a synthetic event object
attrs_event = SyntheticEvent(
        name="synthetic_nearshore",
        description = "This is a synthetic nearshore event",
        time=TimeFrame(
            start_time=datetime(2020, 1, 1),
            end_time=datetime(2020, 1, 2),
        ),
        forcings={
            ForcingType.WIND: [
                WindConstant(
                    speed=us.UnitfulVelocity(value=5, units=us.UnitTypesVelocity.mps),
                    direction=us.UnitfulDirection(
                        value=60, units=us.UnitTypesDirection.degrees
                    ),
                )
            ],
            ForcingType.RAINFALL: [
                RainfallConstant(
                    intensity=us.UnitfulIntensity(
                        value=20, units=us.UnitTypesIntensity.mm_hr
                    )
                )
            ],
            ForcingType.DISCHARGE: [
                DischargeConstant(
                    river=RiverModel(
                        name="cooper",
                        description="Cooper River",
                        x_coordinate=595546.3,
                        y_coordinate=3675590.6,
                        mean_discharge=us.UnitfulDischarge(
                            value=5000, units=us.UnitTypesDischarge.cfs
                        ),
                    ),
                    discharge=us.UnitfulDischarge(
                        value=5000, units=us.UnitTypesDischarge.cfs
                    ),
                )
            ],
            ForcingType.WATERLEVEL: [
                WaterlevelSynthetic(
                    surge=SurgeModel(
                        timeseries=TimeseriesFactory.from_args(
                            shape_type=ShapeType.triangle,
                            duration=us.UnitfulTime(
                                value=1, units=us.UnitTypesTime.days
                            ),
                            peak_time=us.UnitfulTime(
                                value=8, units=us.UnitTypesTime.hours
                            ),
                            peak_value=us.UnitfulLength(
                                value=1, units=us.UnitTypesLength.meters
                            ),
                        )
                    ),
                    tide=TideModel(
                        harmonic_amplitude=us.UnitfulLength(
                            value=1, units=us.UnitTypesLength.meters
                        ),
                        harmonic_period=us.UnitfulTime(
                            value=12.4, units=us.UnitTypesTime.hours
                        ),
                        harmonic_phase=us.UnitfulTime(
                            value=0, units=us.UnitTypesTime.hours
                        ),
                    ),
                )
            ],
        },
    )

💾 You created your first `Event` object! Let's **save** it in the database. 

In [ ]:
# Save the event to the database
fa.save_event(attrs_event)

## **Step 3**: 📈 Projections - Create a projection

**Projections** in FloodAdapt allow us to adjust our model to future conditions such as sea level rise or/and population growth. If you want to learn more about projections in FlooAdapt please go to the section ["Projections"](https://deltares-research.github.io/FloodAdapt/4_user_guide/projections/) in the FloodAdapt documentation. 

In this example we will create a sea level rise (SLR) projection of 2ft.  
The projections can be divided into two categories:
1. **Physical Projections**: Sea level rise, intensified precipitation, increased storm frequency
2. **Socio economic change**: Population growth (existing built area, new development area), economic growth

When creating a projection we need to create a `Projection` object with the following inputs:  
`name`: The name of the projection  
`description` (optional): The description of the projection  
`PhysicalProjection`: An object that captures the pysical projection  
`SocioEconomicChange`: An object that captures the socio economic change.

In this example we will create the following `PhysicalProjection`:  
`sea_level_rise`: Define the value of the slr and the unit (m, ft)  
`subsidence`: Define the value of the subsidence and the unit (m,ft)  

In this example we will create the following `SocioEconomicChange`:  
`None`: Leave the object empty

To get a deeper understanding for all the possible projections and their inputs go to the notebook [**projectionblablab**](nolink).


In [ ]:
# Create a projection object
attrs_projection = Projection(
    name="SLR_2ft",
    description = "This is a 2ft SLR projection",
    physical_projection=PhysicalProjection(
        sea_level_rise=us.UnitfulLength(value=2, units=us.UnitTypesLength.feet),
        subsidence=us.UnitfulLength(value=1, units=us.UnitTypesLength.feet),
    ),
    socio_economic_change=SocioEconomicChange(),
)

💾 You created your first `Projection` object! Let's **save** it in the database. 

In [ ]:
# Save the projection
fa.save_projection(attrs_projection)

## **Step 4**: 🧱 Measures - Create a measure

**Measures** in FloodAdapt enable the user to investigate the efficiency of these mitigation measures on the spot. Measures can be hydraulic, green infrastructure or be on the building level. You can read more about measures in the section ["Measures"](https://deltares-research.github.io/FloodAdapt/4_user_guide/measures/) in the FloodAdapt documentation. 

In this example we will create a **hydraulic measure**, a sea wall of 12ft.  

When creating a sea wall as measure we need to create a `Measure` object with the following inputs:  
`name`: The name of the measure  
`description` (optional): The description of the measure  
`type`: The type of the measure. In this example a floodwall object  
`selection type`: The type of shape. In this example we define a polyline for the shape of the seawall  
`polygon file`: The file path to the spatial file of the seawall  
`elevation`: The elevation of the sea wall. Define a value and the unit (ft,m)  


In [ ]:
# Create a measure object
attrs_measure_phy = FloodWall(
        name= "Seawall_12ft",
        description = "12ft Seawall",
        type = MeasureType.floodwall,
        selection_type = SelectionType.polyline,
        polygon_file = str(r"C:\Users\rautenba\repos\Database\charleston_full\input\measures\seawall\seawall.geojson"),
        elevation = us.UnitfulLength(value=12, units=us.UnitTypesLength.feet))

💾 You created your first `Measure` object! Let's **save** it in the database. 

In [ ]:
# Save the measure
fa.save_measure(attrs_measure_phy)

Let's add another measure on the **building level**. We can elevate buildings in a specific area to mititgate the impact on these assets.

When elevating biuldings as measure we need to create a `Measure` object with the following inputs:
`name`: The name of the measure  
`description` (optional): The description of the measure  
`type`: The type of the measure. In this example an Elevate object.  
`selection type`: The type of shape. In this example we define a polygone for selecting the area of affected buildings  
`polygon file`: The file path to the spatial file of the to be elevated area  
`property type`: The type of building that sholud be elevated. Define the building type (residnetial, commercial...) of your Fiat Model you wish to elevate. In this example we want to elevate all buildings.  
`elevation`: The elevation of the buildings. Define a value, the unit (ft,m) and the vertical reference for the elevation (datum, floodmap).  

In [ ]:
# Create a measure object
attrs_measure_elev = Elevate(
        name= "Elevated_homes",
        description = "Elevate residential buildings",
        type = MeasureType.elevate_properties,
        selection_type = SelectionType.polygon,
        polygon_file = str(r"C:\Users\rautenba\repos\Database\charleston_full\input\measures\raise_property_polygon\raise_property_polygon.geojson"),
        property_type = "ALL",
        elevation = us.UnitfulLengthRefValue(value=1, units=us.UnitTypesLength.feet, type = us.VerticalReference.floodmap))

# Create the measure
fa.create_measure(attrs_measure_elev.model_dump(exclude_none=True), type=attrs_measure_elev.type)

# Save the measure
fa.save_measure(attrs_measure_elev)

## **Step 5**: 🧩 Strategies - Create a strategy

**Strategies** in FloodAdapt are a combination of measures. When we compile multiple measures in a strategy, we first have to create all te measures we would like to use. Find out more about strategies in the section ["Strategies"](https://deltares-research.github.io/FloodAdapt/4_user_guide/strategy/) in the FloodAdapt documentation. 

In this example we will create a strategy with the measures we created above, a **sea wall** of 12ft and **elevating homes** 1ft above the floodmap reference in a specific area.  

When creating a strategy we need to create a `Strategy` object with the following inputs:  
`name`: The name of the strategy  
`description` (optional): The description of the strategy  
`measures`: A list of all the measure object names, you would like to use in that strategy   

In [ ]:
# Create a strategy object
attrs_strategy = Strategy(
        name= "seawall_and_elev_build",
        description = "Strategy with a seawall and elevation of buildings",
        measures = [attrs_measure_phy.name, attrs_measure_elev.name],
        )

💾 You created your first `Strategy` object! Let's **save** it in the database. 

In [ ]:
# Save the stategy
fa.save_strategy(attrs_strategy )

## **Step 6**:🗺️ Create a scenario

We reached the final step where we can put all the building blocks together to create a complete scenario!  
A scenario is composed of:

**1. Event**  
**2. Projection**  
**3. Strategy (Measures)**

If you want to read more about the composition of scenarios, go read the [**Scenario**](https://deltares-research.github.io/FloodAdapt/4_user_guide/scenarios/)-section of the FloodAdapt documentation. 

In this example we will create a scenario with **the event, projection and strategy** we created above. 
When creating a scenario we need to create a `Scenario` object with the following inputs:  
`name`: The name of the strategy  
`description` (optional): The description of the scenario  
`event`: The name of the event object   
`projection`: The name of the projection object   
`strategy`: The name of the strategy object  

In [ ]:
# Create a scenario object
scenario = Scenario(
    name = "slr_nearshore_seawall_elev_build",
    description = "Nearshore event with SLR projection and seawall + elevated buildings strategy",
    event =  attrs_event.name,
    projection =  attrs_projection.name,
    strategy = attrs_strategy.name,
)

💾 You created your first `Scenario` object! Let's **save** it in the database. 

In [ ]:
# Save the scenario
fa.save_scenario(scenario)

## **Final step**: 🏃‍♀️ Run a scenario

We are ready to **run** the scenario! Simply use the **API** to run the scenario.

In [ ]:
# Run the scenario
fa.run_scenario(scenario.name)

### **Finished!** 
**Congratulations** you created and ran your first FloodAdapt scenario!

## **Output**: 🕵️‍♀️ Let's inspect the output

#### **1. Output files**
In your scenario output folder you should see the following files:
- **Flooding**: Folder
- **Impacts**: Folder
- **finished.txt**: text file
- **Infometrics_"*scenario_name*".csv**: csv file of the overall infometrics
- **Infometrics_"*scenario_name*"_"*aggregation_layer*".csv**: csv file  of the aggregated areas. You have one file per aggregation level. In this example we have two files. 
- **logfile_"*scenario_name*".log**: The log of the scenario run
- **"*scenario_name*"_metrics.html**: A metric file of your scenario output

The figure below presents a visual overview of all the output files that should be in your database after running the scenario
<div>
<img src="../_static/images/output_folder_event.png" width="500"/>
</div>

#### **2. Floodmap** - Inspect the floodmap

We can open and inspect the floodmap geotiff.

In [ ]:
root = Path("../_data/examples/charleston_test").resolve()

# Open floodmap geotiff
data_name = root / "output"/ "scenarios" / "slr_nearshore_seawall_elev_build" / "Flooding" / "FloodMap_slr_nearshore_seawall_elev_build.tif"
tiff = rasterio.open(data_name)

# Plot floodmap
rasterio.plot.show(tiff, title = "Floodmap nearshore scenario with 2ft SLR, seawall & elevated buildings", cmap="Blues")

#### **3. Economic Impacts** - Inspect the economic impacts on the building level and aggregated

We can plot the economic impacts on the building level and on the aggregated level.

In [ ]:
## Building Impacts
# Open building impacts
gdf_impacts_buildings = gpd.read_file(root / "output" / "scenarios" / "slr_nearshore_seawall_elev_build" / "Impacts" / "Impacts_building_footprints_slr_nearshore_seawall_elev_build.gpkg")

# Reproject buildings crs to Web Mercator
gdf_impacts_buildings =gdf_impacts_buildings.to_crs(epsg=3857)

# Plot building impacts
ax = gdf_impacts_buildings.plot(figsize=(10, 10),column = "Total Damage",cmap="Reds", legend = True, vmin= 0, vmax= 60000, legend_kwds={"label": "Total Damages ($) Buildings","orientation": "horizontal"})
cx.add_basemap(ax)
ax.plot()

## Aggregated Impacts
# Open aggregated impacts
gdf_impacts_aggr = gpd.read_file(root / "output" / "scenarios" / "slr_nearshore_seawall_elev_build" / "Impacts" / "Impacts_aggregated_slr_nearshore_seawall_elev_build_aggr_lvl_2.gpkg")

# Reproject buildings crs to Web Mercator
gdf_impacts_aggr = gdf_impacts_aggr.to_crs(epsg=3857)
# Plot aggregated impacts
ax = gdf_impacts_aggr.plot(figsize=(10, 10),column = "TotalDamageEvent",cmap="Reds", legend = True, vmin= 0, vmax= 10000000, edgecolor="k", legend_kwds={"label": "Total Damages ($) per aggregatetion area", "orientation": "horizontal"})
cx.add_basemap(ax)
ax.plot()

#### **4. Infometrics & Infographics**
Which **infometrics** and **infographics** to generate can be defined in the infometrics andd infographics and configuration file in your database */Database/charleston_full/static/templates/infometrics/""*, */Database/charleston_full/static/templates/infographics/"".toml*, respectively.

The figure below shows the infographics of the scenario we created above. 
<div>
<img src="../_static/images/example_chaleston3.png" width="600"/>
</div>